In [ ]:
import tweepy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import os
import unicodedata
import math
from apps import *
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [40]:
def autenticar(n_app):

    n_app = n_app % 259
    app = apps[n_app]
    consumer_key = app['consumer_key']
    consumer_secret = app['consumer_secret']
    access_token = app['access_token']
    access_token_secret = app['access_token_secret']
    

    auth = tweepy.OAuthHandler(consumer_key=consumer_key, consumer_secret=consumer_secret)

    auth.set_access_token(access_token, access_token_secret)

    # Creation of the actual interface, using authentication
    api = tweepy.API(auth)

    #Error handling
    if (not api):
        print ("Problem Connecting to API")
    else:
        print "Autenticado " + str(n_app)

    #print api.rate_limit_status()
    return api


def autenticar_app(n_app):

    n_app = n_app % 255
    app = apps[n_app]
    consumer_key = app['consumer_key']
    consumer_secret = app['consumer_secret']
    access_token = app['access_token']
    access_token_secret = app['access_token_secret']

    #Switching to application authentication
    auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)

    #Setting up new api wrapper, using authentication only
    api = tweepy.API(auth) #wait_on_rate_limit=True
     
    #Error handling
    if (not api):
        print ("Problem Connecting to API")
    else:
        print "Autenticado App" + str(n_app)
    return api

def buscar_tweets(prov,api,n_app,printi,cant_tweets):
    
    tweet_count = 0
    cant_tweets = cant_tweets
    coords = provincias[prov]['coords']
    
    f_tweets = open(prov+'_tweets.json','a')
    f_users = open(prov+'_users.json','a')
    usr_prov = 0
    cant_por_coord = 40000
    
    while (usr_prov < cant_tweets):
        for coord in coords:
            if usr_prov >= cant_tweets:
                #print 'break2'
                break

            ic = 0
            #print str(coords.index(coord)) +  '/' + str(len(coords))
            prev = 0
            
            #print 'coord, tweet' ,coord, tweet_count
            #print "Intento " + str(tweet_count)
            act = 0 
            try:
                for tweet in tweepy.Cursor(api.search, count=100,lang="es",geocode=(coord + ',20mi')).items():
                    #print(tweet.id)
                    tweet_count += 1
                    ic += 1
                    if tweet_count % printi == 0:
                        print '\t' , tweet_count, ic, usr_prov, prev
                        if prev and usr_prov - prev < 2:
                            break
                        prev = usr_prov

                    

                    if tweet.user.location!="":  
                        location=unicodedata.normalize('NFKD',tweet.user.location).encode('ASCII', 'ignore').lower().replace('-',' ').replace(';',' ').replace(',',' ').replace('|',' ').replace('?',' ').replace('¿',' ').replace("\ ",' ').replace('/',' ')
                        location = word_tokenize(location)
                        #print location
                        words = provincias[prov]['words']
                        matches=[x for x in location if x in words]
                        if len(matches)>0 and tweet.user.id not in ids:
                            usr_prov += 1
                            
                            f_users.write(json.dumps(tweet._json['user']))
                            f_users.write("\n")
                            f_tweets.write(json.dumps(tweet._json))
                            f_tweets.write("\n")
                            ids[tweet.user.id] = 1
                            location = tweet.user.location
                            loc[location] = 1 if not loc.has_key(location) else loc[location] +1
                            if usr_prov >= cant_tweets:
                                #print 'break2'
                                break
                            
                    if ic >= cant_por_coord:
                        #print 'break1'
                        break
                            
            except tweepy.TweepError,e:
                n_app += 1
                #print "Error " + str(e)
                api = autenticar_app(n_app)
                continue
            except Exception, e:
                #print "Error " + str(e)
                continue

    f_tweets.close()
    f_users.close()
    return n_app



In [6]:

import json
i = 0
file_path = 'ult_json/jujuy_users.json'
with open(file_path) as f:
    for line in f:
        content = json.loads(line)
        print i,content['id_str']
        i += 1

0 1182881023 ElissLlampa
1 4079004855 BarbiVildosola
2 2564389912 florchinadiez
3 2731680862 delfimeedina
4 1972736695 magamonaldi
5 702154495292260355 candpalavecinoo
6 1134625434 danyelcastro82
7 1575547994 paulalamasok
8 194331755 AnttoMartinnez
9 732765399872344064 periodicociudad
10 2743808327 NocetiAnita
11 138533086 juanjo2580
12 2416582975 RociiAguirre15
13 3420966676 TeAmoRiveeeer
14 715957865463353344 romicaba510
15 2173123681 AntoMaurizzio_
16 834292567 conti_tucu
17 90401187 CarolinaMoises
18 3171881231 aldanalucas26
19 69415618 WorldGrace
20 2985039761 AyelenColque
21 3731676675 lia96anto
22 197146723 Cristinajuy
23 4878837148 Ayluu_Airbag
24 69348116 laramontial
25 273921778 Santycheto92
26 148862991 ch3eEcHuU
27 157478600 NeneDeAntes_38
28 1328242513 LasMutulas
29 795280933104586752 HebeDianaLpez13
30 3352733560 TomiiRusitto
31 159679983 FlorciCejas
32 208612931 OriOve
33 2881782857 mickagutierrez1
34 17951477 fado82
35 2279964512 SangreJujuy
36 630511341 JuliArrayaOk
37

In [49]:
from apps import *
arch = 'ult_json/' + prov + '_users.json'

with open(arch) as f:
    num_lines = sum(1 for line in f) / 2
    print prov,num_lines

sanluis 1000


In [66]:
import tweepy
import calendar
file_path = 'ult_json/' + 'jujuy' + '_users.json'
api = autenticar_app(0)
i= 0 
num = 3
with open(file_path) as f:
    for line in f:
       
        
    print elid
    for t in tweepy.Cursor(api.user_timeline, id=elid,count = 200, include_rts = False ).items():
        my_date =  t.created_at
        
        print my_date,my_date.weekday(),calendar.day_name[my_date.weekday()]

Autenticado App0
1182881023
4079004855
2564389912
2564389912
2017-02-12 18:25:26 6 Sunday
2017-02-12 18:24:32 6 Sunday
2017-02-12 17:45:27 6 Sunday
2017-02-11 15:56:19 5 Saturday
2017-02-11 03:45:04 5 Saturday
2017-02-11 03:18:33 5 Saturday
2017-02-10 04:31:16 4 Friday
2017-02-10 04:26:14 4 Friday
2017-02-10 04:24:32 4 Friday
2017-02-10 00:28:54 4 Friday
2017-02-09 17:29:28 3 Thursday
2017-02-09 17:01:04 3 Thursday
2017-02-09 16:59:18 3 Thursday
2017-02-09 04:48:18 3 Thursday
2017-02-09 04:47:52 3 Thursday
2017-02-09 00:58:22 3 Thursday
2017-02-08 23:28:16 2 Wednesday
2017-02-08 18:07:37 2 Wednesday
2017-02-08 16:39:07 2 Wednesday
2017-02-08 16:38:27 2 Wednesday
2017-02-08 16:36:45 2 Wednesday
2017-02-08 16:35:34 2 Wednesday
2017-02-08 16:16:25 2 Wednesday
2017-02-08 16:09:07 2 Wednesday
2017-02-08 00:17:14 2 Wednesday
2017-02-07 18:57:11 1 Tuesday
2017-02-07 17:03:38 1 Tuesday
2017-02-07 03:31:03 1 Tuesday
2017-02-07 03:13:26 1 Tuesday
2017-02-07 03:09:02 1 Tuesday
2017-02-07 03:05:28

In [67]:
print t.created_at.weekday()

4


In [63]:
import tweepy
import calendar
file_path = 'ult_json/' + 'jujuy' + '_users.json'
api = autenticar_app(0)
i= 0 
num = 10
with open(file_path) as f:
    for line in f:
        if i < num:
            i+=1
            elid = json.loads(line)['id']
            print elid
        else:
            break
        

Autenticado App0
1182881023
4079004855
2564389912
2731680862
1972736695
702154495292260355
1134625434
1575547994
194331755
732765399872344064


In [8]:
import json
file_path = 'ult_json/' + 'mendoza' + '_users.json'
with open(file_path) as f:
    for line in f:
        user = json.loads(line)
        print user['location']

General Alvear, Mendoza
General Alvear, Mendoza
Bowen Mendoza
General Alvear, Mendoza
General Alvear, Mendoza
General Alvear - Mendoza 
Mendoza, General Alvear
General Alvear mendoza 
General Alvear, Mendoza
General Alvear-Mendoza
Mendoza Gral. Alvear
Gral Alvear, Mendoza
Mendoza, Argentina
General Alvear-Mendoza
General Alvear- Mendoza
Mendoza 
General Alvear, Mendoza 
General alvear, Mendoza 
General Alvear, Mendoza
Bowen Mendoza
General Alvear-Mendoza
General Alvear, Mendoza 
Bowen, Mendoza 
General Alvear, Mendoza
Villa Atuel
Bowen,Mendoza, Argentina
General Alvear, Mendoza
General Alvear Mendoza
General Alvear Mza Argentina
Mendoza, Argentina
General Alvear - Mza.
General Alvear,Mendoza
General Alvear, Mendoza
General Alvear, Mendoza
Gral Alvear-Mendoza-Argentina 
General Alvear-Mendoza
General Alvear-Mendoza 
General Alvear Mendoza
Bowen, Mendoza, Argentina
San Rafael, Mendoza 
General Alvear Mendoza 
General Alvear Mendoza
San Rafael, Argentina
General Alvear - Mendoza 
General 